# Urban Observatory weather radar data

### Notes
- API is down and radar is currently under maintenence so this code has not been tested.
- Radar processing based on typical radar workflow, for more information see https://wradlib.org/.
- Might need a few exceptions putting on it - don't anticipate radar will be working again before end of PYRAMID project.

### What does this code do
1. Downloads raw Urban Observatory X-band weather radar data from the radar API.
2. Removes static ground clutter.
3. Removes additional clutter (Gabella 2016 filter).
4. Corrects attenuation (Kraemer 2008).
5. Converts to rainfall rates using standard $ZR$-relationship.
6. Reprojects data from polar to Cartesian grid using Nearest Neighbour.
7. Saves raw rainfall data (high resolution in time).
8. Accumulates up the rainfall data to  15mins.

### Outputs format
- `\root` folder path 
    - `\UO` folder path for all Urban Observatory data
        - `\radar` folder path for Urban Observatory X-band radar rainfall data (processed)
            - `arrays.npy` - radar data arrays (t, y, x) 
            - `timestamp.csv`- radar data timestamp
            - `coords_x.csv` - radar data x-coordinates
            - `coords_y.csv` - radar data y-coordinates
            - `\15min` folder path (15 minute X-band radar data)
                - `arrays.npy` - radar data arrays (t, y, x)
                - `timestamp.csv`- radar data timestamp
                - `coords_x.csv` - radar data x-coordinates
                - `coords_y.csv` - radar data y-coordinates

In [3]:
import os
import pandas as pd
import requests
from os.path import join, exists
import numpy as np
import io
import shutil
import wradlib as wrl
import matplotlib.pyplot as plt

In [4]:
# file paths to change
root_path = r"C:\Users\Amy\OneDrive - Newcastle University (1)\Documents\PYRAMID\data\realtime"
out_path = join(join(root_path, "UO"), "radar")

if not exists(out_path):
    os.mkdir(out_path)

    os.environ["WRADLIB_DATA"] = out_path

static_path = join(root_path, "static")
static_clutter = np.load(join(static_path, "static_clutter_map.npy")) 

In [12]:
##### Inputs to change
start_date = "2022-06-20"
end_date = "2022-06-30"

# Bounding box for data 
e_l, n_l, e_u, n_u = [355000, 534000, 440000, 609000]
bbox = [e_l, e_u, n_l, n_u]
elevation_angle = 3
api_date_string_format = "%Y%m%d%H%M%S"
params = {
        'starttime': pd.to_datetime(start_date).strftime(api_date_string_format),
        'endtime': pd.to_datetime(end_date).strftime(api_date_string_format),
        'angle': elevation_angle
    }

In [ ]:
raw_names = []
timestamp = []
tabs = []
  
try:
    # get list of tiff file names of radar images within time range
    r = requests.get("https://api.radar.urbanobservatory.ac.uk/raw", params)

    raw_names.extend([obj["hdf_name"] for obj in r.json()["results"]])
    timestamp.extend([obj["timestamp"] for obj in r.json()["results"]])
    tabs.append(pd.DataFrame(r.json()["results"]))
except:
    print(date)
        
hdf_tab = pd.concat(tabs)

if len(tabs) > 0:
    
    # download file data 
    for f in hdf_tab.hdf_name:
        
        r = requests.get("https://api.radar.urbanobservatory.ac.uk/raw/file/" + f + "/", stream=True)

        if r.status_code == 200:
            
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            # Open a local file with wb ( write binary ) permission.
            with open(join(root_path, f), 'wb') as file:
                shutil.copyfileobj(r.raw, file)
            

In [44]:
# gridding data information
radar_location = (-1.6145999431610107, 54.98040008544922, 96.0)  # (lon, lat, alt) in decimal degree and meters
radar_location_bng = (424764 , 565156, 96)
elevation = 1.5  # in degree
azimuths = np.arange(0, 360)  # in degrees
ranges = np.arange(0, 240) * 150  # in meters
polargrid = np.meshgrid(ranges, azimuths)

coords, rad = wrl.georef.spherical_to_xyz(
    polargrid[0], polargrid[1], elevation, radar_location
)
x = coords[..., 0]
y = coords[..., 1]

bng = wrl.georef.epsg_to_osr(27700)
proj_coords = wrl.georef.reproject(coords, projection_source=rad,
                                  projection_target=bng)

x = proj_coords[..., 0]
y = proj_coords[..., 1]

xgrid = np.arange(x.min(), x.max() + 125, 250)
ygrid = np.arange(y.min(), y.max() + 125, 250)
grid_xy = np.meshgrid(xgrid, ygrid)
grid_xy = np.vstack((grid_xy[0].ravel(), grid_xy[1].ravel())).transpose()

xy = np.concatenate([x.ravel()[:, None], y.ravel()[:, None]], axis=1)

C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\wradlib\georef\projection.py:206: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal.VersionInfo("RELEASE_NAME")) < LooseVersion("3"):
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\wradlib\georef\projection.py:206: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal.VersionInfo("RELEASE_NAME")) < LooseVersion("3"):
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\wradlib\georef\projection.py:304: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal.VersionInfo("RELEASE_NAME")) >= LooseVersion("3"):
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\wradlib\georef\projection.py:304: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal.VersionInfo("RELEASE_NAME")) >= L

In [69]:
file_names = [f for f in os.listdir(root_path) if f.endswith(".z")] 
file_names.sort()

rainfall = []
ts = []

for f in file_names:

    try:
        # read in data
        fcontent = wrl.io.read_opera_hdf5(wrl.util.get_wradlib_data_file(join(root_path, f)))
        nodata = fcontent["dataset1/data1/what"]["nodata"]
        offset = fcontent["dataset1/data1/what"]["offset"]
        gain = fcontent["dataset1/data1/what"]["gain"]
        rrange = fcontent['dataset1/where']["rscale"] * fcontent['dataset1/where']["nbins"]
        data = fcontent['dataset1/data1/data'] * gain + offset

        # remove static clutter
        data_no_static = wrl.ipol.interpolate_polar(data, static_clutter.astype(np.uint8))

        # additional clutter filter
        gabella_clutter = wrl.clutter.filter_gabella(data_no_static, wsize=5, thrsnorain=0., tr1=6., n_p=8, tr2=1.3)
        data_no_clutter = wrl.ipol.interpolate_polar(data_no_static, gabella_clutter)

        # correct attenuation
        pia = wrl.atten.correct_attenuation_constrained(
            data_no_clutter, 
            a_max=1.67e-4, a_min=2.33e-5, n_a=100,
            b_max=0.7, b_min=0.65, n_b=6,
            #a_max=1.94e-4, a_min=1e-6, n_a=100, 
            #b_max=0.9, b_min=0.65, n_b=10,
            gate_length=0.150,
            constraints=[wrl.atten.constraint_dbz, wrl.atten.constraint_pia],
            constraint_args=[[59.0], [20.0]])
        data_attcorr = data_no_clutter + pia
        rain = wrl.zr.z_to_r(wrl.trafo.idecibel(data_attcorr))

        gridded = wrl.comp.togrid(xy, grid_xy, 36000.0, np.array([x.mean(), y.mean()]),
            rain.ravel(), wrl.ipol.Nearest)
        gridded_rain = np.ma.masked_invalid(gridded).reshape((len(xgrid), len(ygrid)))

        rainfall.append(gridded_rain)
        ts.append(pd.to_datetime(f.split("-")[3]))
    except:
        print("Reading in failed.")
    
    
rainfall = np.stack(rainfall)

data_timestamp = pd.Series(np.arange(len(ts)), index=ts).astype(str)
data_timestamp = data_timestamp.apply(lambda x: ''.join(x + ','))
data_timestamp = data_timestamp.tz_localize("utc")

# save high temporal resolution radar data
pd.Series(data_timestamp.index).to_csv(join(out_path, "timestamp.csv"), index=False)
np.save(join(out_path, "arrays.npy"), rainfall.data)
pd.Series(xgrid).to_csv(join(out_path, "coords_x.csv"), index=False)
pd.Series(ygrid).to_csv(join(out_path, "coords_y.csv"), index=False)

out_path_15min = join(out_path, "15min")
if not exists(out_path_15min):
    os.mkdir(out_path_15min)
    
# new 15 minute timestamp
new_timestamp = pd.date_range(
    pd.to_datetime(start_date),
    pd.to_datetime(end_date) + pd.Timedelta(1, "d"),
    freq=str(15 * 60) + "s", 
    tz="UTC"
)

# resample the data
new_array_indices = pd.Series(",", index=new_timestamp)
resampled = data_timestamp.astype(str).resample(str(60*15) + "s").sum()

new_array_indices.loc[resampled.index] = resampled 

new_arrays = np.full((len(new_timestamp), rainfall.data.shape[1], rainfall.data.shape[2]), np.nan)

for i in range(len(new_timestamp)):
    
    idxs = new_array_indices.iloc[i].split(",")[0:-1]
    if len(idxs) > 1:
        new_arrays[i] = np.nanmean(rainfall.data[np.array(idxs).astype(int)], axis=0)
        
pd.Series(data_timestamp.index).to_csv(join(out_path_15min, "timestamp.csv"), index=False)
np.save(join(out_path_15min, "arrays.npy"), rainfall.data)
pd.Series(xgrid).to_csv(join(out_path_15min, "coords_x.csv"), index=False)
pd.Series(ygrid).to_csv(join(out_path_15min, "coords_y.csv"), index=False)

C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\wradlib\atten.py:331: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  forward_large_sectors[iii] = ndimage.morphology.binary_dilation(
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\wradlib\atten.py:334: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  backward_large_sectors[iii] = ndimage.morphology.binary_dilation(
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\wradlib\atten.py:331: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  forward_large_sectors[iii] = ndimage.morphology.binary_dilation(
C:\Users\Amy\anaconda3\envs\wradlib\lib\site-packages\wradlib\atten.py:334: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimag